In [31]:
import csv
from bs4 import BeautifulSoup as bs

In [32]:
from selenium import webdriver

In [33]:
from selenium.webdriver.chrome.service import Service
ser = Service("C:\\Program Files\\geckodriver.exe")


In [34]:
driver = webdriver.Firefox(service = ser)

In [35]:
url = 'https://www.amazon.com'
driver.get(url)

In [36]:
def get_url(search_term):
    """Generate a url from a search term"""
    template = 'https://www.amazon.com/s?k=womens+clothes&crid=1V890I551MG9P&sprefix={}%2Caps%2C272&ref=nb_sb_noss'
    search_term = search_term.replace(" ", "+")
    return template.format(search_term)

In [37]:
url = get_url("women's clothes")
print(url)

https://www.amazon.com/s?k=womens+clothes&crid=1V890I551MG9P&sprefix=women's+clothes%2Caps%2C272&ref=nb_sb_noss


In [38]:
driver.get(url)

# Extract the Collection

In [39]:
soup = bs(driver.page_source, 'html.parser')

In [41]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

# ProtoType the record

In [88]:
item = results[2]

Description

In [100]:
description = item.h2.a.text.strip()
description

"Morhuduck Women's V Neck Hoodies Long Sleeve Sweatshirt Drawstring Pullover Tops with Pocket"

URL

In [101]:
url = 'https://www.amazon.com' + atag.get('href')
url

'https://www.amazon.com/Morhuduck-Hoodies-Sweatshirt-Drawstring-Pullover/dp/B09BVM5MB3/ref=sr_1_3?crid=1V890I551MG9P&keywords=womens+clothes&qid=1644300524&sprefix=women%27s+clothes%2Caps%2C272&sr=8-3'

Price

In [92]:
price_parent = item.find('span', 'a-price')

In [93]:
price = price_parent.find('span', 'a-offscreen').text
price

'$21.99'

Stars / 5

In [94]:
rating_parent = item.find('i')

In [95]:
rating = stars_parent.find('span', 'a-icon-alt').text
rating

'4.3 out of 5 stars'

Reviews

In [109]:
reviews_count = item.find('span', {'class' : 'a-size-base'}).text
reviews_count

'217'

# Generalise the pattern

In [112]:
def extract_record(item):
    """extract and return data from a single record"""
    
    # description 
    description = item.h2.a.text.strip()
    
    # url
    url = 'https://www.amazon.com' + atag.get('href')
    
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    # rating / 5
    rating_parent = item.find('i')
    rating = stars_parent.find('span', 'a-icon-alt').text
    
    # reviews (total amount)
    reviews_count = item.find('span', {'class' : 'a-size-base'}).text
    
    result = (description, url, price, rating, reviews_count)
    
    return result

In [113]:
records = []
results = results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'find'

# Error Handling

### Some listing do not have all attributes

In [124]:
def extract_record(item):
    """extract and return data from a single record"""
    
    # description 
    description = item.h2.a.text.strip()
    
    # url
    url = 'https://www.amazon.com' + atag.get('href')
    
    # price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    # rating / 5
    try:
        rating_parent = item.find('i')
        rating = stars_parent.find('span', 'a-icon-alt').text
    except AttributeError:
        rating = ""
    
    # reviews (total amount)
    try:
        reviews_count = item.find('span', {'class' : 'a-size-base'}).text
    except AttributeError:
        reviews_count = ""
    
    result = (description, url, price, rating, reviews_count)
    
    return result

In [125]:
records = []
results = results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(extract_record(item))

In [126]:
records[0]

('ROSKIKI Womens Crochet Hollow-Out Sleeve Button Down Ribbed Knitted Pullover Tops',
 'https://www.amazon.com/Morhuduck-Hoodies-Sweatshirt-Drawstring-Pullover/dp/B09BVM5MB3/ref=sr_1_3?crid=1V890I551MG9P&keywords=womens+clothes&qid=1644300524&sprefix=women%27s+clothes%2Caps%2C272&sr=8-3',
 '$21.99',
 '4.3 out of 5 stars',
 '834')

# Get the next page and all 20 pages (amazon max)

In [132]:
def get_url(search_term):
    """Generate a url from a search term"""
    template = 'https://www.amazon.com/s?k=womens+clothes&crid=1V890I551MG9P&sprefix={}%2Caps%2C272&ref=nb_sb_noss'
    search_term = search_term.replace(" ", "+")
    
    # add term query to url
    url = template.format(search_term)
    
    # add page number
    url += '&page={}'
    
    return template.format(search_term)

# putting it all together

In [141]:
import csv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service



def get_url(search_term):
    """Generate a url from a search term"""
    template = 'https://www.amazon.com/s?k=womens+clothes&crid=1V890I551MG9P&sprefix={}%2Caps%2C272&ref=nb_sb_noss'
    search_term = search_term.replace(" ", "+")
    
    # add term query to url
    url = template.format(search_term)
    
    # add page number
    url += '&page={}'
    
    return template.format(search_term)

def extract_record(item):
    """extract and return data from a single record"""
    
    # description 
    description = item.h2.a.text.strip()
    
    # url
    url = 'https://www.amazon.com' + atag.get('href')
    
    # price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    # rating / 5
    try:
        rating_parent = item.find('i')
        rating = stars_parent.find('span', 'a-icon-alt').text
    except AttributeError:
        rating = ""
    
    # reviews (total amount)
    try:
        reviews_count = item.find('span', {'class' : 'a-size-base'}).text
    except AttributeError:
        rewviews_count = ""
    
    result = (description, price, rating, reviews_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    # startup the web driver
    ser = Service("C:\\Program Files\\geckodriver.exe")
    driver = webdriver.Firefox(service = ser)
    
    records = []
    
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = bs(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    # save to csv
    with open('all_results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
    
    

In [142]:
main('computer monitor')